# **Práctica de Reconocimiento de Formas**

- **Alumno 1**: Carlos Jiménez Martín - 160190
- **Alumno 2**: Enrique González Rodríguez - 160329

# **1. Introducción**
En el presente documento se expone la práctica final que corresponde a la primera mitad de la asignatura de Reconocimiento de Formas. 

Primero se exponen el clasificador de la distancia euclídea y el estadístico bayesiano que han sido implementados a lo largo de la duración de esta primera parte.

Otro apartado corresponde a la regularización de los datos, en el que se expone el algoritmo usado para poder llevar a cabo la propia regularización de las matrices de covarianzas.

El siguiente apartado corresponde a las técnicas de evaluación de rendimiento que se han estudiado en la asignatura y su utilización para evaluar los clasificadores implementados anteriormente.

Finalmente se presenta un caso real de reconocimiento de formas, un OCR, el cual se implementa mediante uno de los clasificadores implementados anteriormente y probando su precisión en diferentes escenarios.

# **2. Clasificador de la distancia Euclídea**

El clasificador de la distancia Euclídea es un clasificador que se basa en calcular la distancia mínima del punto que se quiere clasificar a los respresentaantes de cada clase, escogiendo la clase que minimize dicha distancia.

Para la implementación de este clasificador se ha utlizado la libreria _numpy_, y se ha realizado de forma que funcione de forma correcta para un problema de clasificación que involucre un número cualquiera de clases, datos y elementos de vector de características. Se ha utlizado el método llamado **_broadcasting_** para que las matrices con con tamaños diferentes puedan operarse de la manera más óptima posible.

Se ha utlizado:
- **_np.mean_** para calcular los centroides de cada clase.
- **_np.linalg.norm_** para calcular la distancia euclídea de los puntos a clasificar, ante los representantes de cada clase.
- **_np.argmin_** para obtener la mínima distancia entre el dato a clasificar y una clase.

En la fase **_fit_** se han calculado los centroides de las clases.

En la fase **_predict_** se calcula la distancia del punto a clasificar a los centroides de las diferentes clases, escogiendo la clase cuya distancia sea la menor. 

In [0]:
import numpy as np
from abc import abstractmethod

class Classifier:

    @abstractmethod
    def fit(self,X,y):
        pass

    @abstractmethod
    def predict(self,X):
        pass

class ClassifEuclid(Classifier):
    def __init__(self,labels=[]):
        """Constructor de la clase
        labels: lista de etiquetas de esta clase"""
        self.labels=labels
        self.centroides = None
        pass

    def fit(self,X,y):
        """Entrena el clasificador
        X: matriz numpy cada fila es un dato, cada columna una medida
        y: vector de etiquetas, tantos elementos como filas en X
        retorna objeto clasificador"""
        
        self.centroides = np.array([np.mean(X[y == i], axis=0)for i in labels])
        return self

    def predict(self,X):
        """Estima el grado de pertenencia de cada dato a todas las clases 
        X: matriz numpy cada fila es un dato, cada columna una medida del vector de caracteristicas. 
        Retorna una matriz, con tantas filas como datos y tantas columnas como clases tenga
        el problema, cada fila almacena los valores pertenencia de un dato a cada clase""" 
        
        return np.linalg.norm(self.centroides[:, np.newaxis] - X, axis=2)
    
    def pred_label(self,X):
        """Estima la etiqueta de cada dato. La etiqueta puede ser un entero o bien un string.
        X: matriz numpy cada fila es un dato, cada columna una medida
        retorna un vector con las etiquetas de cada dato"""
        
        return self.labels[np.argmin(X, axis=0)]
    
    def num_aciertos(self,X,y):
        """Cuenta el numero de aciertos del clasificador para un conjunto de datos X.
        X: matriz de datos clasificados
        y: vector de etiquetas correctas"""
        
        assert self.centroides is not None, "Error: Debes entrenar primero el clasificador"
        return (X == y).sum(), (X == y).mean() * 100

### **Resultados**

| Base de datos | Número de aciertos | Porcentaje de aciertos |
| --- | --- | --- |
| **Iris**   | 139 | 92.667 |
| **Wine**   | 129 | 72.472 |
| **Cancer** | 507 | 89.104 |
| **MNIST**  | 48479 | 80.798 |
| **Isolet** | 6843 | 87.765 |

El mayor resultado alcanzado es el que corresponde al dataset _Iris_, lo cual puede deberse a que la distancia entre las distintas clases puede identificarse fácilmente debido a que están correctamente dispersas. 

El menor resultado, a su vez, es el del dataset _Wine_. Al contrario que en _Iris_, una menor dispersión de las clases puede hacer que estas distancias sean más dudosas al estar las clases más solapadas entre sí, lo que hace que el clasificador prediga peor los resultados.

# **3. Clasificador Estadístico Bayesiano**

El clasificador estadístico Bayesiano es un clasificador que tiene en cuanta la dispersión y la forma de las clases, es decir, modela la distribución de las muestras de una clase, al contrario que el clasificador de la distancia euclídea. Este clasificador sigue el teorema estadístico de Bayes, donde se cumple que: 

- $P(\alpha_i|x) = \frac{P(x|\alpha_i)\cdot P(\alpha_i)}{P(x)}$

siendo _$\alpha_i$_ una clase del modelo de datos, y _x_ el dato a clasificar.

El criterio de clasificación se basa en minimizar la probabilidad de error del clasificador, de manera que se elije aquella clase cuya probabilidad a posteriori sea máxima.

La fórmula general del clasificador estadístico bayesiano para una clase con distribución gausiana es:

- $f_i(x) = \mathcal{N}(x|\mu_i,Σ_i)\cdot P(\alpha_i)$

Tomando $d_i(x) = \ln{f_i(x)}$:

- $d_i(x) = -\frac{1}{2}\ln{|Σ_i|} - \frac{1}{2}(x-\mu_i)^\intercalΣ_i^{-1}(x-\mu_i)+\ln{P(\alpha_i)}$

donde $Σ_i$ es la matriz de covarianzas de la clase _i_, que muestra mediante una matriz cuadrada la relación de unas características con otras a través de sus varianzas y covarianzas.

Para su implementación en Python se ha utilizado:
-  **_np.log_** para calcular los logaritmos neperianos de las probabilidades a priori y de las matrices de covarianzas.
- **_np.mean_** para calcular las medias de cada clase.
- **_np.cov_** para calcular las matrices de covarianzas.
- **_np.linalg.inv_** para calcular la inversa de las matrices de covarianzas.
- **_np.matmul_** para multiplicar matrices.
- **_np.expand_dims_** para añadir dimensiones a las matrices.
- **_np.squeeze_** para eliminar dimensiones de las matrices.
- **_np.unique_** para obtener los elementos únicos de un array.

Se ha utilizado _broadcasting_ y compresión de listas para poder implementar el algoritmo.

En la fase **_fit_** primero se calculan los logaritmos neperieanos de la probabilidad a priori de cada clase, despué se calcula el valor de la media _$\mu$_ para cada clase. Seguidamente se calculan las matrices de covarianzas _$Σ$_ de cada clase y sus logaritmos neperianos, y finalmente se calcula la inversa de estas.

En la fase **_predict_**, únicamente se calcula el grado de pertenencia de cada valor a cada clase, a partir de los parámetros calculados en el entrenamiento, mediante la formula $d_i(x)$ que se muestra mas arriba.

In [0]:
import numpy as np
from abc import abstractmethod

class Classifier:

    @abstractmethod
    def fit(self,X,y):
        pass

    @abstractmethod
    def predict(self,X):
        pass

class ClassifBayesiano(Classifier):
    def __init__(self):
        """Constructor de la clase
        labels: lista de etiquetas de esta clase"""
        self.labels = None
        self.ln_apriories = None
        self.means = None
        self.ln_determinants = None
        self.inv_covs = None
        
    def fit(self,X,y):
        """Entrena el clasificador. Dado que es un clasificador Gausiano Bayesiano, 
        se aprenderán los parámetros de las gausianas de cada clase.
        X: matriz numpy cada fila es un dato, cada columna una característica
        y: vector de etiquetas, tantos elementos como filas en X
        retorna objeto clasificador"""
        assert X.ndim == 2 and X.shape[0] == len(y)
        
        # Contar cuantos ejemplos hay de cada etiqueta
        self.labels, labels_count = np.unique(y, return_counts=True)
        
        # Usando el contador de ejemplos de cada etiqueta, calcular el logaritmo neperiano de las probabilidades a-priori
        self.ln_apriories = np.log(labels_count/y.size)
        
        # Calcular para los ejemplos de cada clase, la media de cada una de sus características (centroide)
        self.means = np.array([np.mean(X[y == i], axis=0) for i in np.unique(y)])
        
        # Sustraer a los ejemplos de cada clase su media y calcular su matriz de covarianzas (puedes emplear compresión de listas) 
        self.covs = np.array([np.cov(X[y==np.unique(y)[i]]-self.means[i], rowvar=False) for i in range(len(self.labels))])
        
        # Para cada una de las clases, calcular el logaritmo neperiano de su matriz de covarianzas (puedes emplear compresión de listas o la función map)
        self.ln_determinants = np.log(np.linalg.det(self.covs))
        
        # Para cada una de las clases, calcular la inversa de su matriz de covarianzas (puedes emplear compresión de listas o la función map)
        self.inv_covs = np.linalg.inv(self.covs)

        return self

    def predict(self,X):
        """Estima el grado de pertenencia de cada dato a todas las clases 
        X: matriz numpy cada fila es un dato, cada columna una medida del vector de caracteristicas. 
        Retorna una matriz, con tantas filas como datos y tantas columnas como clases tenga
        el problema, cada fila almacena los valores pertenencia de un dato a cada clase""" 
        assert self.means is not None, "Error: The classifier needs to be fitted. Please call fit(X, y) method."
        assert X.ndim == 2 and X.shape[1] == self.means.shape[1]

        # Resta la media de cada clase a cada ejemplo en X
        X_mean0 = np.array([X-self.means[i] for i in range(len(self.means))])

        a = -(1/2)*self.ln_determinants
        b = np.matmul(np.expand_dims(X_mean0, 2), np.expand_dims(self.inv_covs,1))
        c = - (1/2)*np.matmul(b,  np.expand_dims(X_mean0, 3))

        # Calcula el logaritmo de la función de decisión gausiana 
        # Transparencias de Métodos paramétricos de clasificación: página 14:
        # -(1/2)ln|Sigma_i| - (1/2)*(x- mu_i)^T Sigma_i^-1 (x- mu_i) + lnP(alpha_i)
        grado_de_pertenencia = -(1/2)*self.ln_determinants - (1/2)*np.squeeze(np.matmul(np.matmul(np.expand_dims(X_mean0, 2), np.expand_dims(self.inv_covs,1)),  np.expand_dims(X_mean0, 3))).T + self.ln_apriories
        return grado_de_pertenencia 

    def pred_label(self,X):
        """Estima la etiqueta de cada dato. La etiqueta puede ser un entero o bien un string.
        X: matriz numpy cada fila es un dato, cada columna una medida
        retorna un vector con las etiquetas de cada dato"""
        print("X shape", X.shape)
        return self.labels[np.argmax(X, axis=1)]

### **Resultados**

| Base de datos | Número de aciertos | Porcentaje de aciertos |
| --- | --- | --- |
| Iris   | 147 | 98.0 |
| Wine   | 177 | 99.4382 |
| Cancer | 554 | 97.3637 |
| MNIST  | - | - |
| Isolet | - | - |


Como se puede observar en los resultados, se alcanzan mejores porcentajes en este caso que con el clasificador de la distancia euclídea, menos para los dataset MNIST e Isolet, dónde no se puede aplicar el clasificador estadístico bayesiano. 

Como ambos dataset cuentan con un número elevados de parámetros respecto al número de datos, ocurre un error de matriz singular (no invertible) al calcularse la inversa de la matriz de covarianzas en el entrenamiento, haciendo imposible el uso de este clasificador sin algún método de regularización.

# **4. Regularización**

Cuando en el dataset que utilizamos tiene pocos datos y un número elevado de características discriminantes, se comparte la matriz de covarianzas para todas las clases, ya que la función discriminante lineal $Σ_i = Σ_j$ puede dar mejor rendimiento, debido a que emplean menos parámetros.

En el caso de que se dispongan muy pocos datos, se puede asumir que $Σ_i = Σ_j = \sigma^2I$, donde $I$ es la matriz identidad, en cuyo caso el clasificador resultante es bastante similar al de la distancia euclídea.

El proceso de modificar las matrices de covarianzas para mejorar el rendimiento de nuestro clasificador se denomina regularización, y es adecuado cuando se dispone de pocos datos y un número elevado de parámetros. Se dispone de dos parámetros:
- $\lambda$: el grado de igualdad entre las $Σ_i$
- $\gamma$: el grado de semejanda a la matriz identidad de las $Σ_i$.

Se deben seleccionar los valores de $\lambda$ y $\gamma$ que optimizan el rendimiento del clasificador.

Para la implementación de la **regularización** en el clasificador estadístico bayesiano se han utilizado los parámetros:
- **share_covs**: indica si se comparte la matriz de covarianzas entre las clases o no.
- **shrinkage**: determina el grado de diagonalidad de la matriz de covarianzas.



In [0]:
import numpy as np
from sklearn.covariance import ShrunkCovariance
from sklearn import preprocessing
from abc import abstractmethod
import pandas as pd
import numpy as np


class Classifier:

    @abstractmethod
    def fit(self,X,y):
        pass

    @abstractmethod
    def predict(self,X):
        pass

class ClassifBayesianoParametrico(Classifier):
    def __init__(self, share_covs=False, shrinkage=0.0):
        """Constructor de la clase
        share_covs: Indica si la matriz de covarianzas va a ser compartida entre las distintas clases.
        shrinkage: Parámetro que determina la diagonalidad de la matriz de covarianzas. Ver sklearn.covariance.ShrunkCovariance
        """
        assert 0 <= shrinkage <= 1
        self.labels = None
        self.ln_apriories = None
        self.means = None
        self.ln_determinants = None
        self.inv_covs = None
        self.share_covs = share_covs
        self.shrinkage = shrinkage
        self.scaler = preprocessing.StandardScaler()

    def fit(self, X, y):
        """Entrena el clasificador
        X: matriz numpy cada fila es un dato, cada columna una medida
        y: vector de etiquetas, tantos elementos como filas en X
        retorna objeto clasificador"""
        assert X.ndim == 2 and X.shape[0] == len(y)
        # Aseguramos que las etiquetas son numeros tal que: [0, 1, ..., N]
        y = pd.factorize(y)[0]
        # Preprocesamos los datos de entrada
        X = self.scaler.fit_transform(X)
        # Contar cuantos ejemplos hay de cada etiqueta
        self.labels, labels_count = np.unique(y, return_counts=True)
        # Usando el contador de ejemplos de cada etiqueta, calcular el logaritmo neperiano de las probabilidades a-priori
        self.ln_apriories = np.log(labels_count/y.size)
        # Calcular para los ejemplos de cada clase, la media de cada una de sus características (centroide)
        self.means = np.array([np.mean(X[y == i], axis=0) for i in np.unique(y)])
        
        if self.share_covs:
            # Restamos al dato de cada clase su centroide
            xmz = np.array([])
            for l in self.labels:
              xmz = np.append(xmz.reshape(-1, X.shape[1]), X[y==l] - self.means[l], axis=0)
            # Calcula la matriz de covarianzas empleando la clase ShrunkCovariance de sklearn
            cov =  ShrunkCovariance(shrinkage=self.shrinkage).fit(xmz).covariance_
            #cov = ShrunkCovariance(shrinkage=self.shrinkage).fit(xmz).get_precision()
            # La reproducimos tantas veces como número de clases
            covs = np.tile(cov, (len(self.labels), 1, 1))
            print("Covarianzas conjuntas: ", covs, covs.shape)
        else:
            # Calcula la matriz de covarianzas empleando la clase ShrunkCovariance de sklearn
            covs = np.array([ShrunkCovariance(shrinkage=self.shrinkage).fit(X[y==np.unique(y)[i]]-self.means[i]).covariance_ for i in range(len(self.labels))])
            print("Covarianzas por separado: ", covs, covs.shape)

        
        # Para cada una de las clases, calcular el logaritmo neperiano de su matriz de covarianzas (puedes emplear compresión de listas o la función map)
        self.ln_determinants = np.log(np.linalg.det(covs))
        if np.any(np.isinf(self.ln_determinants)):
          print("Warning: There is a covariance matrix with determinant equal zero")
        # Para cada una de las clases, calcular la inversa de su matriz de covarianzas (puedes emplear compresión de listas o la función map)
        self.inv_covs = np.linalg.inv(covs)
        
        return self

    def predict(self, X):
        """Estima el grado de pertenencia de cada dato a todas las clases
        X: matriz numpy cada fila es un dato, cada columna una medida del vector de caracteristicas.
        Retorna una matriz, con tantas filas como datos y tantas columnas como clases tenga
        el problema, cada fila almacena los valores pertenencia de un dato a cada clase"""
        assert self.means is not None, "Error: The classifier needs to be fitted. Please call fit(X, y) method."
        assert X.ndim == 2 and X.shape[1] == self.means.shape[1]

        # Preprocesamos nuestros datos
        X = self.scaler.fit_transform(X) 

        # Resta la media de cada clase a cada ejemplo en X
        X_mean0 = np.array([X-self.means[i] for i in range(len(self.means))])
        
        # Calcula el logaritmo de la función de decisión gausiana
        # Transparencias de Métodos paramétricos de clasificación: página 14:
        # -(1/2)ln|Sigma_i| - (1/2)*(x- mu_i)^T Sigma_i^-1 (x- mu_i) + lnP(alpha_i)
        grado_de_pertenencia = -(1/2)*self.ln_determinants - (1/2)*np.squeeze(np.matmul(np.matmul(np.expand_dims(X_mean0, 2), np.expand_dims(self.inv_covs,1)),  np.expand_dims(X_mean0, 3))).T + self.ln_apriories
        return grado_de_pertenencia

    def pred_label(self, X):
        """Estima la etiqueta de cada dato. La etiqueta puede ser un entero o bien un string.
        X: matriz numpy cada fila es un dato, cada columna una medida
        retorna un vector con las etiquetas de cada dato"""
        return self.labels[np.argmax(X, axis=1)]

### **Resultados:**

| Base de datos | Número de aciertos | Porcentaje de aciertos |
| --- | --- | --- |
| Iris   | 147 | 98.0 |
| Wine   | 178 | 100 |
| Cancer | 555 | 97.53953 |
| MNIST  | 52289 | 87.14833 |
| Isolet | 7488 | 96.03693 |

Como se puede observar en la tabla, los resultado se ven mejorados respecto a la implementación del clasificador estadístico bayesiano sin regularización. 

Además, al compartir la matriz de covarianzas en las bases de datos MNIST e Isolet se consigue evitar el error obtenido previamente, haciendo las matrices de covarianzas invertibles a través de este tipo de transformación. 

Los resultados obtenidos se deben a los siguiente valores de los parámetros:

| Base de datos | share_covs | shrinkage |
| --- | --- | --- |
| Iris   | False | 0 |
| Wine   | True | 0 |
| Cancer | False | 0 |
| MNIST  | True | 0,3 |
| Isolet | True | 0,25 |



# **5. Evaluación del Rendimiento**

## **5.1 Validación Cruzada**

Para los dataset Iris, Wine y Cancer se ha evaluado el rendimiento del clasificador de la distancia euclídea y el calsificador estadísitico bayesiano con regularización, mediante **validación cruzada _k-fold_**, que consiste en dividir el dataset en k porciones iguales, entrenandolo con k-1 porciones de datos, y evaluandolo con la porcion de datos restante, cambiando en cada iteración la porción del dataset utilizada para entrenar.

Se ha utilizado la función **_cross_val_score_** con **k = 5** para su implementación en Python en el caso del clasificador euclídeo, midiendo la precisión (accuracy) media para cada dataset.

### **Resultados:**

- **Iris:**
  - Clasificador de la distancia euclídea:
    - Accuracy: 0.91 (+/- 0.14)
- **Wine:**
  - Clasificador de la distancia euclídea:
    - Accuracy: 0.73 (+/- 0.12)
- **Cancer:**
  - Clasificador de la distancia euclídea:
    - Accuracy: 0.89 (+/- 0.02)


Como se puede observar todos los resultados son inferiores a los obtenidos mediante el uso del clasificador bayesiano. Esta disminución de la precisión de nuestros clasificadores se debe a que se ha entrenado y probado los clasificadores cada vez con un conjunto de datos distinto en vez de con el mismo conjunto de datos.


## **5.2 Wrapper**

Para mejorar los parámetros externos del clasificador bayesiano, tuvimos que recurrir a los métodos Wrapper. En este caso específico, utilizamos  un método al que llamamos **_best_shrinkage_**, el cual prueba los distintos valores de los parámetros externos **_share_covs_**, para que las clases compartan o no la matriz de covarianzas, y **_shrinkage_**, para definir la diagonalización de la matriz de covarianzas. 

Este proceso parte de unas listas de valores y de los datos de entrenamiento de la base de datos que se quiere probar que se le pasa como argumentos de entrada. Mediante la función **_GridSearchCV_**, se aplica a su vez el concepto del apartado anterior de validación cruzada, obteniendose la precisión media de las distintas combinaciones de conjuntos de datos y eligiendo los parámetros para la más óptima.

### **Resultados:**

- **Iris:**
  - Clasificador estadístico bayesiano regularizado:
    - Accuracy: 0.973 (+/- 0.013)
    - shrinkage = 0
    - share_covs = False
- **Wine:**
  - Clasificador estadístico bayesiano regularizado:
    - Accuracy: 0.955 (+/- 0.023)
    - shrinkage = 0
    - share_covs = True
- **Cancer:**
  - Clasificador estadístico bayesiano regularizado:
    - Accuracy: 0.961 (+/- 0.013)
    - shrinkage = 0.5
    - share_covs = True

En este caso se puede observar una situación similar a la de antes al obtenerse la precisión media, solo que a su vez se elige el mejor valor de los parámetros externos (share_covs y shrinkage) comparando las precisiones obtenidas para todas las combinaciones y quedándose con la mayor media.

## **5.3 Exclusión**

Para los dataset Isolet y MNIST, debido al tamaño de la base datos, se utiliza exclusión como método de evaluación, que significa que se cogen unos subconjuntos de datos del dataset, en vez de utilizar todo el dataset para realizar la validación cruzada, ya que al ser conjuntos de datos tan grandes aplicar este concepto conllevaría un excesivo tiempo de computo.

### **Resultados:**

- **Isolet:**
  - Clasificador de la distancia euclídea:
    - Accuracy: 0.847
  - Clasificador estadístico bayseiano regularizado:
    - Accuracy: 0.939 (+/- 0.000)
    - shrinkage = 0.3
    - share_covs = True
- **MNIST:**
  - Clasificador de la distancia euclídea:
    - Accuracy: 0.820
  - Clasificador estadístico bayseiano regularizado:
    - Accuracy: 0.873 (+/- 0.000)
    - shrinkage = 0.2
    - share_covs = True

En este caso se puede observar como la precisión de los clasificadores ha aumentado respecto a la calculada para el dataset completo en secciones anteriores. Esto se debe a que, a pesar dividir el dataset, al obtenerse el mejor valor de los parámetros para cada caso, la precisión aumenta respecto al uso exclusivo del clasificador bayesiano regularizado.



In [0]:
def best_shrinkage_clf(X, y, k, shrinkages, share_covs):
    """
    Busca el clasificador bayesiano regularizado con el mejor shrinkage. 
    :param X: Ejemplos de la dase de datos
    :param y: Etiquetas de los ejemplos
    :param k: Número de divisiones en la validación cruzada (k-fold)
    :param shrinkages: Lista de posibles shrinkages que conforman la rejilla de búsqueda
    :param share_covs: Lista de posibles valores para share_covs que conforman la rejilla de búsqueda
    """
    from sklearn.model_selection import GridSearchCV
    cbp = ClassifBayesianoParametrico(share_covs)
    params = {'shrinkage': shrinkages, 'share_covs': share_covs}
    clf = GridSearchCV(cbp, params, n_jobs=-2, scoring='accuracy', cv=k).fit(X, y)
    best_clf = clf.best_estimator_
    # print("Srinkage scores: ", clf.cv_results_['mean_test_score'])
    result_score_mean = clf.cv_results_['mean_test_score'][clf.best_index_]
    result_score_std = clf.cv_results_['std_test_score'][clf.best_index_]
    print("\tSelected shrinkage = {}, share_covs = {}\n" \
          "\tAccuracy: {:.3f} (+/- {:.3f})".format(best_clf.shrinkage,
                                                   best_clf.share_covs,
                                                   result_score_mean,
                                                   result_score_std))
    return best_clf

# **6.  Aplicación en un caso real de reconocimiento de texto**

En esta entrega se aplicó el conocimiento adquirido en las anteriores para, a partir de un conjunto de datos generado por nosotros, usar los clasificadores para reconocer texto de una imagen real. Los cambios realizados sobre el esqueleto del código proporcionado se muestran en los siguientes apartados.

### **6.1 Clasificador**

Para este caso, se ha decidido usar el clasificador de la distancia euclídea, ya que los resultados obtenidos eran óptimos. 

### **6.2 Pipeline**

A la hora de entrenar el clasificador, se ha utilizado un pipeline al que se le añaden varios filtros de selección de características: 
- VarianceThreshold:  Descarta los datos con varianza 0, en este caso alrededor de 300 características.
- SelectKBest: Descarta el 5% de las características del dataset que son peores, para que no influyan en el entrenamiento del clasificador.


In [0]:
n_vt_cols = VarianceThreshold().fit(X_train, y_train).transform(X_train).shape[1]
clf = Pipeline([('feature_selection1',VarianceThreshold()), 
                ('feature_selection2',SelectKBest(chi2, k=round(n_vt_cols*0.95))),
                ('classification',ClassifEuclid())])

### **6.3 Tamaño de imagen y texto**

Para optimizar la lectura de la imagen se amplió su anchura en píxeles de 2500 a 5000, de manera que ésta fuera más nítida y fácil de reconocer, aunque requiriera más computo.




In [0]:
IMG_WIDTH = 5000.0

Después, adaptamos parámetros del código para que fueran acordes a este cambio, como el contorno mínimo para reconocer las letras (de 120 a 500).

In [0]:
contours = contours[np.array(list(map(cv2.contourArea, contours))) > 500]

También se ha cambiado el grosor de los carácteres para hacer que se separen mejor unas letras de otras y no se reconozcan varias como un solo carácter.

In [0]:
bin_img = cv2.dilate(bin_img, np.ones((2, 2), np.uint8))

Por último, en la parte donde se busca la separación de las letras para evitar reconocer varios carácteres como uno solo, se ha añadido un bucle extra para realizar una doble comprobación, obteniendose mejores resultados. Además, se han eliminado los carácteres que están por debajo de un área definida, en este caso por debajo de 0.66.

In [0]:
        for box in char_boxes:
            n_splits = round((box[2] / box[3]) / CHAR_ASPECT_RATIO)
            if n_splits > 1:
                # We have detected a box that clearly contains two chars
                char_boxes.remove(box)
                for i in range(n_splits):
                    char_boxes.append([int(box[0] + (box[2] / n_splits) * i), box[1], int(box[2] / n_splits), box[3]])
            elif n_splits < 0.66:
                char_boxes.remove(box)
                

        for box in char_boxes:
            n_splits = round((box[2] / box[3]) / CHAR_ASPECT_RATIO)
            if n_splits > 1:
                # We have detected a box that clearly contains two chars
                char_boxes.remove(box)
                for i in range(n_splits):
                    char_boxes.append([int(box[0] + (box[2] / n_splits) * i), box[1], int(box[2] / n_splits), box[3]])
            elif n_splits < 0.66:
                char_boxes.remove(box)

### **6.4 Dataset**

Se ha modificado la base de datos inicial, añadiendo el carácter "/" al no estar previamente y aparecer en el texto. Se probó a añadir los carácteres de interrogación, pero nos empeoraba el reconocimiento más de lo que lo mejoraba al reconocer otros carácteres como interrogaciones, por lo que decidimos no incluirlos en el diccionario de carácteres.

Además, se ha ampliado el número de datos generados para entrenar y testear, de manera que el clasificador tenga más datos de entrada y logre un rendimiento más óptimo. Como observación, en el grupo de los datos de entrenamiento se han metido datos con ruido y datos sin ruido, de manera que el conjunto de datos de entrenamiento sea lo más variado posible. 


In [0]:
labels = np.array(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H',
                   'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
                   '/'
                   ])

font_filename = os.path.join(resources_dir, "AndaleMono.ttf")
X_train1, y_train1 = generate_ttf_images(font_filename, 90, labels, True)
X_train2, y_train2 = generate_ttf_images(font_filename, 10, labels, False)
X_train = np.append(X_train1, X_train2, axis=0)
y_train = np.append(y_train1, y_train2)
X_train = X_train.reshape(len(X_train), -1).astype(float)

print("--> Generating test dataset")
X_test, y_test = generate_ttf_images(font_filename, 25, labels, True)
X_test = X_test.reshape(len(X_test), -1).astype(float)

Los parámetros de la función que añade ruido al conjunto de datos de entrenamiento han sido elegidos de manera que, el conjunto quede lo más variado posible para que se pueda adaptar a varias imagenes de entrada con diferentes ángulos.

In [0]:
 if add_noise==True:
              # Add noise
              img = add_noise_to_img(img, rot_noise_level=2, scale_noise_level=0.04, gauss_noise_level=3, blur_noise_level=3, salt_level=1, dilate_level=6)


Finalmente, se ha modificado el tamaño en píxeles de las imagenes generadas para entrenar y testear el clasificador, reduciendo el ancho de la misma de 28 píxeles a 25.

In [0]:
    @staticmethod
    def standardize_char_size(img: np.ndarray, dst_shape: tuple = (28, 25)):
        """
        Centers and scales the input image (img) to fit it in a image of shape dst_shape with a small border.
        :param img: The input image
        :param dst_shape: The destination shape. This defines also the number of features used in the classifier.
        :return: a new image where the input image has been scaled and centered.
        """
        dst = np.zeros(dst_shape, dtype=float)
        h, w = img.shape
        margin = 2
        # Scale the image to fit in the destination shape
        scale_factor = (dst_shape[0] - 2 * margin) / h if h > w else (dst_shape[1] - 2 * margin) / w
        resized = cv2.resize(img, (int(scale_factor * w), int(scale_factor * h)))
        # Convert from white background to black background
        resized = 255 - resized.astype(float)
        # Paste in the output image
        x_margin = (dst_shape[1] - resized.shape[1]) // 2
        y_margin = (dst_shape[0] - resized.shape[0]) // 2
        dst[y_margin:y_margin + resized.shape[0], x_margin:x_margin + resized.shape[1]] = resized
        return dst

### **6.5 Filtros**

Se han amplicado varios filtros a la imagen de entrada, de manera que fuera lo más limpia y nítida posible para facilitar el reconocimiento de ésta posteriormente:

-GaussianBlur: Aplica un filtro de suavizado a la imagen a partir de un kernel pasado como parámetro de entrada

-Threshold: Aplica un filtro que fija un umbral binario a partir del cual se elige el pixel de color blanco, de manera que convierte la parte gris externa de los carácteres a blanco, para que la diferencia entre el fondo de la imagen y el texto sea lo más clara posible.

In [0]:
# Blur image to remove some noise
blur = cv2.GaussianBlur(img, (3, 3), 1)

# Binarize image using the smart Otsu binarization method
bin_img = cv2.threshold(img, 170, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

# **7.  Conclusiones**

En esta parte de la asignatura, dirigida especialmente a esta práctica, hemos aprendido multitud de conceptos acerca de la parte de Reconocimiento de Formas dentro del campo de la Inteligencia Artificial, muy útiles de cara a un futuro dónde el manejo de técnicas de análisis de datos y su clasificación está cada vez más en auge y más demandado en el mercado. 

La mezcla de realizar tareas más prácticas tanto a mano como en código han servido de gran ayuda para entender los distintos conceptos, complementándose muy bien entre ellas a partir de los conocimientos adquiridos previamente en clase. Además, ha sido muy útil el feedback recibido tanto en clase como vía e-mail y en tutorías, resolviendose todas nuestras dudas de una manera rápida y precisa por parte tanto de Luis como de Iago.

El transcurso de esta práctica se ha hecho muy ameno debido a su dinámica, ya que se empezó desde cero enseñándose los conceptos básicos tanto de Python como de las herramientas que ibamos a usar en este lenguaje (librerías numpy y el paquete de aprendizje scikit-learn), aumentandose la complejidad según se iban realizando entregas y se explicaban los conceptos utilizados en clase. Esto nos ha facilitado mucho seguir de manera semanal la asignatura, consumiendo el tiempo justo en la realización de la práctica y permitiendo trabajar a su vez en el resto de asignaturas sin ningún tipo de problema.

Para concluir, en forma de resumen, hemos quedado muy satisfechos con esta parte de la asignatura por todo lo aprendido en la misma, la fluidez de las clases y la evaluación de las distintas partes muy dirigidas hacía el alumno, facilitando el paso por la asignatura por evaluación contínua.